###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces
import sqlite3 as sq
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
#functions
def percent(x, y):
    return (x/y)*100
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

# This notebook outlines the download and formatting process for the vehicular crash data coming from other research staff. There is minimal formatting this document is more of a placeholder in case there ever is formatting needed.

In [3]:
#import and examine data
data = pd.read_csv('../../Data Downloads/Crashes.csv')
data['Year'] = data['Year'].astype(str)
data

,NAME,Year,Crashes Total,Crashes:Fatal,Crashes:Serious Injury
0,Adams,2013,10,0,0
1,Adams,2014,13,0,1
2,Adams,2015,15,0,1
3,Adams,2016,13,0,0
4,Adams,2017,15,0,1
5,Adams,2018,14,0,1
6,Adams,2019,22,0,0
7,Adams,2020,5,0,0
8,Adams,2021,12,0,1
9,Adams,2022,9,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   NAME                    650 non-null    object
 1   Year                    650 non-null    object
 2   Crashes Total           650 non-null    int64 
 3   Crashes:Fatal           650 non-null    int64 
 4   Crashes:Serious Injury  650 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 25.5+ KB


In [5]:
years = list(data['Year'].unique())

In [6]:
data = data.set_index(['NAME', 'Year']).transpose()

In [7]:
data.head()

NAME                   Adams                                              Antioch                                              Arrington            \
Year                    2013 2014 2015 2016 2017 2018 2019 2020 2021 2022    2013 2014 2015 2016 2017 2018 2019 2020 2021 2022      2013 2014 2016   
Crashes Total             10   13   15   13   15   14   22    5   12    9      16  137  571  538  543  506  796  450  593  580        10   11    1   
Crashes:Fatal              0    0    0    0    0    0    0    0    0    0       0    2    1    1    2    1    2    1    2    3         0    0    0   
Crashes:Serious Injury     0    1    1    0    1    1    0    0    1    0       1    4    8    9   11    6    5    9   10   14         0    0    0   

NAME                                                 Belle Meade                                              Bellevue                           \
Year                   2017 2018 2019 2020 2021 2022        2013 2014 2015 2016 2017 2018 2019 2020 2021 2022     2013 2014 2015 2016 2017 2018   
Crashes Total             2    6    4    1    3    8          68   77   93   87   98  104  106   80   47   54        7    7   17    1    2    1   
Crashes:Fatal             0    0    0    0    0    0           0    0    0    0    0    0    0    0    0    1        0    1    1    1    0    0   
Crashes:Serious Injury    0    0    0    0    0    1           3    5    2    4    1    3    1    1    0    1        1    0    1    0    1    0   

NAME                                       Berry Hill                                              Bethpage                                          \
Year                   2019 2020 2021 2022       2013 2014 2015 2016 2017 2018 2019 2020 2021 2022     2013 2014 2015 2016 2017 2019 2020 2021 2022   
Crashes Total             3    7    6   10        177  193  259  263  264  221  265  149  144  159       15    8    4    3    1    3    4    3   10   
Crashes:Fatal             1    0    0    2          0    0    0    0    0    0    0    0    0    0        0    1    0    0    0    1    0    0    1   
Crashes:Serious Injury    1    0    1    1          1    0    6    6    3    0    3    2    2    2        1    1    0    0    0    0    0    0    0   

NAME                   Brentwood                                                    Castalian Springs                                    Cedar Hill  \
Year                        2013 2014  2015  2016  2017  2018  2019 2020 2021  2022              2013 2014 2015 2016 2017 2020 2021 2022       2013   
Crashes Total                883  977  1579  1551  1510  1433  1447  856  986  1066                11    3    4    1    2    1    6    1          6   
Crashes:Fatal                  1    0     1     0     4     2     2    4    3     3                 0    0    0    0    0    0    0    0          0   
Crashes:Serious Injury        17   34    39    15    30    11     8   14   21    11                 1    0    0    0    0    0    0    1          0   

NAME                                                                Christiana                                              College Grove            \
Year                   2014 2015 2016 2017 2018 2019 2020 2021 2022       2013 2014 2015 2016 2017 2018 2019 2020 2021 2022          2013 2014 2015   
Crashes Total            10   15   14   19   15   16   14   13   16         13    9   10    7    4    5    5    2    5   21             4    4    2   
Crashes:Fatal             0    0    0    0    0    0    0    0    0          0    0    0    0    0    0    0    0    0    1             0    0    0   
Crashes:Serious Injury    0    1    0    1    0    0    0    0    0          1    0    1    0    0    0    0    0    0    0             0    0    0   

NAME                                                      Columbia                                                       Coopertown                 \
Year                   2016 2017 2018 2019 2020 2021 2022     2013  2014  2015  2016  2017  201

In [8]:
test = data[('Adams', '2013')]
test

Crashes Total             10
Crashes:Fatal              0
Crashes:Serious Injury     0
Name: (Adams, 2013), dtype: int64

In [12]:
for year in years:
#     thelist = [data[('Cheatham', '{}'.format(year))], data[('Davidson', '{}'.format(year))], data[('Dickson', '{}'.format(year))],
#                data[('Houston', '{}'.format(year))], data[('Humphreys', '{}'.format(year))], data[('Maury', '{}'.format(year))],
#                data[('Montgomery', '{}'.format(year))], data[('Robertson', '{}'.format(year))], data[('Rutherford', '{}'.format(year))],
#                data[('Stewart', '{}'.format(year))], data[('Sumner', '{}'.format(year))], data[('Trousdale', '{}'.format(year))],
#                data[('Williamson', '{}'.format(year))], data[('Wilson', '{}'.format(year))]]
#     data[('GNRC Region', '{}'.format(year))] = sum(thelist)
#     thelist = [data[('Cheatham', '{}'.format(year))], data[('Davidson', '{}'.format(year))], data[('Dickson', '{}'.format(year))],
#                data[('Houston', '{}'.format(year))], data[('Humphreys', '{}'.format(year))],
#                data[('Montgomery', '{}'.format(year))], data[('Robertson', '{}'.format(year))], data[('Rutherford', '{}'.format(year))],
#                data[('Stewart', '{}'.format(year))], data[('Sumner', '{}'.format(year))], data[('Trousdale', '{}'.format(year))],
#                data[('Williamson', '{}'.format(year))], data[('Wilson', '{}'.format(year))]]
#     data[('GNRC', '{}'.format(year))] = sum(thelist)  
    thelist = [data[("Eagleville", '{}'.format(year))], data[("LaVergne", '{}'.format(year))], 
               data[("Murfreesboro", '{}'.format(year))], data[("Nolensville", '{}'.format(year))], data[("Smyrna", '{}'.format(year))]]
    data[('Rutherford Unincorporated', '{}'.format(year))] = data[("Rutherford", '{}'.format(year))] - sum(thelist)  
    data[('Rutherford Incorporated', '{}'.format(year))] = sum(thelist)
    thelist = [data[('Davidson', '{}'.format(year))], data[('Maury', '{}'.format(year))], data[('Robertson', '{}'.format(year))],
               data[('Rutherford', '{}'.format(year))], data[('Sumner', '{}'.format(year))], data[('Williamson', '{}'.format(year))],
               data[('Wilson', '{}'.format(year))]]
    data[('MPO', '{}'.format(year))] = sum(thelist)  

In [13]:
data = data.transpose().reset_index(drop = False)

In [14]:
#calculate the percentage of all crashes that are fatal or result in serious injury
data['Crashes%:Fatal'] = percent(data['Crashes:Fatal'], data['Crashes Total'])
data['Crashes%:Serious Injury'] = percent(data['Crashes:Serious Injury'], data['Crashes Total'])

In [15]:
#set index to name and transpose to rename geographies using custom module then transpose back and reset index without dropping
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [16]:
data.head()

,NAME,Year,Crashes Total,Crashes:Fatal,Crashes:Serious Injury,Crashes%:Fatal,Crashes%:Serious Injury
0,"Adams city, Tennessee",2013,10,0,0,0.0,0.0
1,"Adams city, Tennessee",2014,13,0,1,0.0,7.692308
2,"Adams city, Tennessee",2015,15,0,1,0.0,6.666667
3,"Adams city, Tennessee",2016,13,0,0,0.0,0.0
4,"Adams city, Tennessee",2017,15,0,1,0.0,6.666667


In [17]:
#make sure year is formatted as an integer
data['Year'] = data['Year'].astype(int)
#create a list of years from the dataframe to pass through our "generate time frames" function to create a list of all possible time frames - need this here for later
years = list(data['Year'].unique().astype(int))
time_frames = generate_time_frames(years)

In [18]:
#create a multilevel column header with year and placeholder for time frames
#pivot the table and create a multiindex of year and column header
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Year')
df_pivot = data.pivot_table(index = 'NAME', columns = ['Year'], values = cols)
df_pivot.head(2)

Crashes Total                                                                Crashes%:Fatal                                \
Year                           2013   2014   2015   2016   2017   2018   2019   2020   2021   2022           2013      2014      2015      2016   
NAME                                                                                                                                              
Adams city, Tennessee          10.0   13.0   15.0   13.0   15.0   14.0   22.0    5.0   12.0    9.0            0.0  0.000000  0.000000  0.000000   
Antioch                        16.0  137.0  571.0  538.0  543.0  506.0  796.0  450.0  593.0  580.0            0.0  1.459854  0.175131  0.185874   

                                                                                  Crashes%:Serious Injury                                          \
Year                       2017      2018      2019      2020      2021      2022                    2013      2014      2015      2016      2017   
NAME                                                                                                                                                
Adams city, Tennessee  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000                    0.00  7.692308  6.666667  0.000000  6.666667   
Antioch                0.368324  0.197628  0.251256  0.222222  0.337268  0.517241                    6.25  2.919708  1.401051  1.672862  2.025783   

                                                                   Crashes:Fatal                                               \
Year                       2018      2019 2020      2021      2022          2013 2014 2015 2016 2017 2018 2019 2020 2021 2022   
NAME                                                                                                                            
Adams city, Tennessee  7.142857  0.000000  0.0  8.333333  0.000000           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
Antioch                1.185771  0.628141  2.0  1.686341  2.413793           0.0  2.0  1.0  1.0  2.0  1.0  2.0  1.0  2.0  3.0   

                      Crashes:Serious Injury                                                  
Year                                    2013 2014 2015 2016  2017 2018 2019 2020  2021  2022  
NAME                                                                                          
Adams city, Tennessee                    0.0  1.0  1.0  0.0   1.0  1.0  0.0  0.0   1.0   0.0  
Antioch                                  1.0  4.0  8.0  9.0  11.0  6.0  5.0  9.0  10.0  14.0

In [19]:
#add a level to the multiindex to accomodate the time period metrics
df_pivot.columns = pd.MultiIndex.from_tuples([(col[0], col[1], 'None') for col in df_pivot.columns])
df_pivot.head(3)

Crashes Total                                                                Crashes%:Fatal                                \
                               2013   2014   2015   2016   2017   2018   2019   2020   2021   2022           2013      2014      2015      2016   
                               None   None   None   None   None   None   None   None   None   None           None      None      None      None   
NAME                                                                                                                                              
Adams city, Tennessee          10.0   13.0   15.0   13.0   15.0   14.0   22.0    5.0   12.0    9.0            0.0  0.000000  0.000000  0.000000   
Antioch                        16.0  137.0  571.0  538.0  543.0  506.0  796.0  450.0  593.0  580.0            0.0  1.459854  0.175131  0.185874   
Arrington                      10.0   11.0    NaN    1.0    2.0    6.0    4.0    1.0    3.0    8.0            0.0  0.000000       NaN  0.000000   

                                                                                  Crashes%:Serious Injury                                          \
                           2017      2018      2019      2020      2021      2022                    2013      2014      2015      2016      2017   
                           None      None      None      None      None      None                    None      None      None      None      None   
NAME                                                                                                                                                
Adams city, Tennessee  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000                    0.00  7.692308  6.666667  0.000000  6.666667   
Antioch                0.368324  0.197628  0.251256  0.222222  0.337268  0.517241                    6.25  2.919708  1.401051  1.672862  2.025783   
Arrington              0.000000  0.000000  0.000000  0.000000  0.000000  0.000000                    0.00  0.000000       NaN  0.000000  0.000000   

                                                                    Crashes:Fatal                                               \
                           2018      2019 2020      2021       2022          2013 2014 2015 2016 2017 2018 2019 2020 2021 2022   
                           None      None None      None       None          None None None None None None None None None None   
NAME                                                                                                                             
Adams city, Tennessee  7.142857  0.000000  0.0  8.333333   0.000000           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
Antioch                1.185771  0.628141  2.0  1.686341   2.413793           0.0  2.0  1.0  1.0  2.0  1.0  2.0  1.0  2.0  3.0   
Arrington              0.000000  0.000000  0.0  0.000000  12.500000           0.0  0.0  NaN  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                      Crashes:Serious Injury                                                  
                                        2013 2014 2015 2016  2017 2018 2019 2020  2021  2022  
                                        None None None None  None None None None  None  None  
NAME                                                                                          
Adams city, Tennessee                    0.0  1.0  1.0  0.0   1.0  1.0  0.0  0.0   1.0   0.0  
Antioch                                  1.0  4.0  8.0  9.0  11.0  6.0  5.0  9.0  10.0  14.0  
Arrington                                0.0  0.0  NaN  0.0   0.0  0.0  0.0  0.0   0.0   1.0

In [20]:
#get a list of the varaibles to loop through by indexing into the first level only of the column headers
first_level = df_pivot.columns.get_level_values(0).unique().tolist()
#remove percentages - don't want change metrics on them
first_level = [item for item in first_level if '%' not in item]

In [21]:
#pass the dataframe, the list of variables, time frames, and years through the "calculate change" function
data = calculate_changes(df_pivot, first_level, time_frames = time_frames, years = years)

In [22]:
#reformat and rename columns
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_1':'Year', 'level_2':'Time Frame'})

In [23]:
#map to geoid and set source name
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'Internal Crash Data'

In [24]:
#examine
data.head()

,NAME,Year,Time Frame,Crashes Total,Crashes Total % Change,Crashes Total Change,Crashes%:Fatal,Crashes%:Serious Injury,Crashes:Fatal,Crashes:Fatal % Change,Crashes:Fatal Change,Crashes:Serious Injury,Crashes:Serious Injury % Change,Crashes:Serious Injury Change,GEO_ID,Source
0,"Adams city, Tennessee",2013,None,10.0,NaN,NaN,0.0,0.000000,0.0,NaN,NaN,0.0,NaN,NaN,1600000US4700200,Internal Crash Data
1,"Adams city, Tennessee",2014,None,13.0,NaN,NaN,0.0,7.692308,0.0,NaN,NaN,1.0,NaN,NaN,1600000US4700200,Internal Crash Data
2,"Adams city, Tennessee",2015,None,15.0,NaN,NaN,0.0,6.666667,0.0,NaN,NaN,1.0,NaN,NaN,1600000US4700200,Internal Crash Data
3,"Adams city, Tennessee",2016,None,13.0,NaN,NaN,0.0,0.000000,0.0,NaN,NaN,0.0,NaN,NaN,1600000US4700200,Internal Crash Data
4,"Adams city, Tennessee",2017,None,15.0,NaN,NaN,0.0,6.666667,0.0,NaN,NaN,1.0,NaN,NaN,1600000US4700200,Internal Crash Data


In [25]:
data['NAME'].unique()

array(['Adams city, Tennessee', 'Antioch', 'Arrington',
       'Belle Meade city, Tennessee', 'Bellevue',
       'Berry Hill city, Tennessee', 'Bethpage',
       'Brentwood city, Tennessee', 'Castalian Springs',
       'Cedar Hill city, Tennessee', 'Christiana', 'College Grove',
       'Columbia city, Tennessee', 'Coopertown town, Tennessee',
       'Cottontown', 'Cross Plains city, Tennessee', 'Culleoka',
       'Davidson', 'Donelson', 'Eagleville city, Tennessee', 'Edenwold',
       'Fairview city, Tennessee', 'Forest Hills city, Tennessee',
       'Fosterville', 'Franklin city, Kentucky',
       'Gallatin city, Tennessee', 'Gladeville',
       'Goodlettsville city, Tennessee', 'Greenbrier town, Tennessee',
       'Hampshire', 'Henderson', 'Hendersonville city, Tennessee',
       'Hermitage', 'Joelton', 'LaVergne', 'Lakewood', 'Lascassas',
       'Lebanon city, Tennessee', 'MPO', 'Madison', 'Maury',
       'Millersville city, Tennessee', 'Milton',
       'Mitchellville city, Tennesse

In [26]:
#export to the SQLite database as only the JobsEQ annual data
conn = sq.connect('../../Outputs/Dem_Transpo_Housing_Collection.db')
data.to_sql('Crashes_Annual', conn, if_exists = 'replace', index = False)

3543